# Differential Expression Analysis

This notebook is a first attempt to automate differential gene expression analysis with Python. 

Gene expression data are read from the .tsv file and imported into Python data structures so that we can select "interesting" genes to submit to Enrichr for enrichment analysis.

We'll follow these steps:
1. Filter the most significant genes (p<0.01)
2. Filter Log2FC (Less and equal to -1 or Great and equal to 1P)
3. Clean up data and maintain only Entrez and Ensemble ID, Log2 FC, p.value cols
4. Rank the genes (largest to  smallest using Log2FC)
5. Past results on Enrichr

In [1]:
# Import necessary Python libraries

# Data analysis
import numpy as np
import pandas as pd

# Bioinformatics
import gseapy as gp

# Graphics and plotting
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2

#import seaborn as sns

Reading in data, and 

In [3]:
filename = '../data/Merged_differential_expression_results.tsv'

dex = pd.read_csv(filename, index_col=0, sep='\t')

In [6]:
# Separate the columns into strings (all gene names, descriptions, ...) and numeric (gene expressions, fold changes, and quality metrics)
all_cols = list(dex.columns)
descr_cols = ['Feature_ID', 'entrezgene', 'external_gene_name', 'gene_biotype', 'external_gene_source', 'description']
numeric_cols = [x for x in all_cols if x not in set(descr_cols)]

# The select samples, fold metrics
sample_cols = [x for x in numeric_cols if 'sample' in x]
linfold_cols = [x for x in numeric_cols if 'linearFC' in x]
logfold_cols = [x for x in numeric_cols if 'logFC' in x]

control_3h_cols = ['sample.'+str(x) for x in range(1,5)]
control_12h_cols = ['sample.'+str(x) for x in range(5,9)]
CC_3h_cols = ['sample.'+str(x) for x in range(9,13)]
CC_12h_cols = ['sample.'+str(x) for x in range(13,17)]
BL21_3h_cols = ['sample.'+str(x) for x in range(17,21)]
BL21_3h_bcols = ['sample.'+str(x) for x in range(21,25)]
LPS_3h_cols = ['sample.'+str(x) for x in range(25,29)]
LPS_21h_cols = ['sample.'+str(x) for x in range(29,33)]

# Associate fold column name with p-value column name
adj_p_value_columns = {
    'BL21_12hr-BL21_3hr_logFC': 'BL21_12hr-BL21_3hr_adj.P.Val',
    'BL21_12hr-control_12hr_logFC': 'BL21_12hr-BL21_3hr_adj.P.Val',
    'BL21_3hr-control_3hr_logFC': 'BL21_3hr-control_3hr_adj.P.Val',
    'CC_12hr-CC_3hr_logFC': 'CC_12hr-CC_3hr_adj.P.Val',
    'CC_12hr-control_12hr_logFC':'CC_12hr-control_12hr_adj.P.Val',
    'CC_3hr-control_3hr_logFC':'CC_3hr-control_3hr_adj.P.Val',
    'control_12hr-control_3hr_logFC':'control_12hr-control_3hr_adj.P.Val',
    'LPS_12hr-control_12hr_logFC':'LPS_12hr-control_12hr_adj.P.Val',
    'LPS_12hr-LPS_3hr_logFC':'LPS_12hr-LPS_3hr_adj.P.Val',
    'LPS_3hr-control_3hr_logFC':'LPS_3hr-control_3hr_adj.P.Val'
}

In [26]:
data_column_name = 'control_12hr-control_3hr_logFC'
pval_column_name = adj_p_value_columns[data_column_name]

data = dex[dex[data_column_name]<0.01]

data = data[['entrezgene', 'external_gene_name', data_column_name]]

data.head()

,entrezgene,external_gene_name,control_12hr-control_3hr_logFC
Feature_ID,,,
ENSG00000000419,8813.0,DPM1,-0.223183
ENSG00000001036,2519.0,FUCA2,0.009324
ENSG00000001167,4800.0,NFYA,-0.353835
ENSG00000001497,81887.0,LAS1L,-0.068961
ENSG00000001617,6405.0,SEMA3F,-0.177985


In [38]:
control_3h = dex[control_3h_cols]
CC_3h = dex[CC_3h_cols]
CC_3h.columns = control_3h.columns

linfc = CC_3h/control_3h
linfc.mean(axis=1)
#logfc = np.log2(control_3h).mean(axis=1)

Feature_ID
ENSG00000000003    1.005444
ENSG00000000419    1.014052
ENSG00000000457    1.013057
ENSG00000000460    0.989905
ENSG00000000971    0.993248
                     ...   
ENSG00000278765    1.054194
ENSG00000278771    0.976257
ENSG00000278791    0.959878
ENSG00000278828    1.248435
ENSG00000278845    0.985022
Length: 12781, dtype: float64

In [20]:
samples = dex[sample_cols]
linearFC = dex[linfold_cols]
logFC = dex[logfold_cols]

In [39]:
linearFC

,BL21_12hr-BL21_3hr_linearFC,BL21_12hr-control_12hr_linearFC,BL21_3hr-control_3hr_linearFC,CC_12hr-CC_3hr_linearFC,CC_12hr-control_12hr_linearFC,CC_3hr-control_3hr_linearFC,control_12hr-control_3hr_linearFC,LPS_12hr-control_12hr_linearFC,LPS_12hr-LPS_3hr_linearFC,LPS_3hr-control_3hr_linearFC
Feature_ID,,,,,,,,,,
ENSG00000000003,-1.083693,-1.251437,-1.053277,1.125714,1.046784,1.019504,1.096377,-1.117166,-1.093200,1.072858
ENSG00000000419,-1.154018,1.042383,1.030518,-1.272784,-1.011159,1.078328,-1.167306,1.046202,-1.407763,1.261712
ENSG00000000457,1.181367,1.220174,1.081920,1.092202,1.087923,1.043408,1.047511,1.071466,1.289054,-1.148508
ENSG00000000460,1.204824,1.013808,-1.043828,1.148473,-1.028581,-1.037577,1.138515,1.050954,1.309525,-1.094438
ENSG00000000971,1.526585,1.432968,1.178472,1.241606,-1.054012,-1.042379,1.255462,1.510934,1.741128,1.089478
...,...,...,...,...,...,...,...,...,...,...
ENSG00000278765,-1.331986,-1.372351,-1.096163,-1.031956,1.024441,-1.006378,-1.063921,-1.728957,-1.372571,-1.340167
ENSG00000278771,1.090348,1.337254,-1.016650,-1.205391,-1.061306,-1.097824,-1.246867,1.088573,-1.067052,-1.073438
ENSG00000278791,1.092596,-1.135030,-1.030278,1.379895,1.016256,-1.128055,1.203685,-1.121864,-1.021577,1.096083


In [40]:
dex['CC_3hr-control_3hr_linearFC']

Feature_ID
ENSG00000000003    1.019504
ENSG00000000419    1.078328
ENSG00000000457    1.043408
ENSG00000000460   -1.037577
ENSG00000000971   -1.042379
                     ...   
ENSG00000278765   -1.006378
ENSG00000278771   -1.097824
ENSG00000278791   -1.128055
ENSG00000278828    1.159340
ENSG00000278845   -1.069733
Name: CC_3hr-control_3hr_linearFC, Length: 12781, dtype: float64

In [9]:
linearFC.describe()

,BL21_12hr-BL21_3hr_linearFC,BL21_12hr-control_12hr_linearFC,BL21_3hr-control_3hr_linearFC,CC_12hr-CC_3hr_linearFC,CC_12hr-control_12hr_linearFC,CC_3hr-control_3hr_linearFC,control_12hr-control_3hr_linearFC,LPS_12hr-control_12hr_linearFC,LPS_12hr-LPS_3hr_linearFC,LPS_3hr-control_3hr_linearFC
count,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000,12781.000000
mean,-0.250345,0.065433,0.224025,-0.028546,0.007485,-0.038655,-0.036111,0.020490,-0.179853,0.095835
std,4.863185,1.448497,5.498581,1.642262,1.128314,1.112691,1.556625,1.325880,3.909590,4.224425
min,-304.468522,-6.117308,-5.509527,-73.723662,-5.944423,-3.908533,-71.784074,-6.627508,-196.493955,-4.182751
25%,-1.183710,-1.131482,-1.099019,-1.156770,-1.073290,-1.073701,-1.149747,-1.113566,-1.191531,-1.146362
50%,-1.009279,1.001360,-1.009151,1.007964,1.001322,-1.003681,1.004514,-1.001521,-1.000683,-1.015457
75%,1.171309,1.141647,1.082278,1.161844,1.072778,1.062797,1.134992,1.106641,1.206101,1.109582
max,9.831060,31.790719,266.542952,4.875140,3.607005,3.574831,6.175203,29.446815,14.526968,259.957115


In [19]:
ctrl_samples = samples[['sample.4','sample.3','sample.2','sample.1']]
ctrl_mean = ctrl_samples.mean(axis=1)

In [20]:
x = samples[['sample.12','sample.11','sample.10','sample.9']]
x_mean = x.mean(axis=1)

In [21]:
fc = ctrl_mean/x_mean

In [27]:
dex['BL21_3hr-control_3hr_linearFC']

0       -1.053277
1        1.030518
2        1.081920
3       -1.043828
4        1.178472
           ...   
12776   -1.096163
12777   -1.016650
12778   -1.030278
12779    1.306796
12780   -1.079191
Name: BL21_3hr-control_3hr_linearFC, Length: 12781, dtype: float64

In [24]:
res

[0        0.994905
 1        0.986437
 2        0.987344
 3        1.010412
 4        1.007101
            ...   
 12776    0.982675
 12777    1.032318
 12778    1.041919
 12779    0.837701
 12780    1.015257
 Length: 12781, dtype: float64,
 0        0.994905
 1        0.986437
 2        0.987344
 3        1.010412
 4        1.007101
            ...   
 12776    0.982675
 12777    1.032318
 12778    1.041919
 12779    0.837701
 12780    1.015257
 Length: 12781, dtype: float64]